**Leitura Streaming Via Socket

In [21]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split


spark = SparkSession \
    .builder \
    .appName("StructuredWordCountToFile") \
    .getOrCreate()

In [22]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .option('includeTimestamp', 'true')\
    .load()

23/01/26 00:01:46 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [23]:
# Divide as linhas recebidas em cada palavra
words = lines.select(
        explode(split(lines.value, ' ')).alias('word'))

In [24]:
# cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
wordCounts = words.groupBy("word").count()

In [25]:
#confere o tipo de dataframe
lines.isStreaming

True

#### Abrir o socket
Para abrir o socket é necessário ter o "nc" instalado e utiliar, no terminal, o seguinte comando ANTES de iniciar a célula anterior

nc -lk 9999

In [27]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado 
query = wordCounts\
    .writeStream\
    .outputMode('update')\
    .format('console')\
    .start()



query.awaitTermination() #aguarda até que a "streaming query" termine 

23/01/26 00:01:51 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/hn/zqmlf1cd4s3_llkx270x0kw80000gn/T/temporary-4bf43871-5d00-46cd-a933-f472836bbdf7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/26 00:01:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|  dados|    1|
|     de|    1|
|ciencia|    1|
+-------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|  dados|    2|
|     de|    2|
|ciencia|    2|
+-------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|dados|    3|
+-----+-----+

23/01/26 00:02:12 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 